<a href="https://colab.research.google.com/github/zulvaf/animelist-goodbooks-recommendation/blob/master/Animelist_Collaborative%20Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Collaborative Filtering

## Library Instalation

In [ ]:
!pip install recommenders surprise cornac==2.3.0 ranx

In [ ]:
!add-apt-repository ppa:ubuntu-toolchain-r/test
!apt-get update
!apt-get install --only-upgrade libstdc++6

## Check Hardware Software Specification

In [ ]:
!python --version

Python 3.11.12


In [ ]:
!pip --version

pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)


In [ ]:
!cat /etc/*release

DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=22.04
DISTRIB_CODENAME=jammy
DISTRIB_DESCRIPTION="Ubuntu 22.04.4 LTS"
PRETTY_NAME="Ubuntu 22.04.4 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.4 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy


In [ ]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: AuthenticAMD
cpu family	: 23
model		: 49
model name	: AMD EPYC 7B12
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2249.998
cache size	: 512 KB
physical id	: 0
siblings	: 8
core id		: 0
cpu cores	: 4
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep_good nopl nonstop_tsc cpuid extd_apicid tsc_known_freq pni pclmulqdq ssse3 fma cx16 sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm cmp_legacy cr8_legacy abm sse4a misalignsse 3dnowprefetch osvw topoext ssbd ibrs ibpb stibp vmmcall fsgsbase tsc_adjust bmi1 avx2 smep bmi2 rdseed adx smap clflushopt clwb sha_ni xsaveopt xsavec xgetbv1 clzero xsaveerptr arat npt nrip_save umip rdpid
bugs		: sysret_ss_attrs null_seg spectre_v1 spectre_v2 spec_store_bypass retbleed smt_rsb srso ibpb

In [ ]:
!cat /proc/meminfo

MemTotal:       53469676 kB
MemFree:        45231484 kB
MemAvailable:   51804888 kB
Buffers:          461940 kB
Cached:          6523968 kB
SwapCached:            0 kB
Active:          1066756 kB
Inactive:        6695960 kB
Active(anon):       6508 kB
Inactive(anon):   777764 kB
Active(file):    1060248 kB
Inactive(file):  5918196 kB
Unevictable:          20 kB
Mlocked:              20 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               812 kB
Writeback:             0 kB
AnonPages:        777184 kB
Mapped:           384788 kB
Shmem:              7436 kB
KReclaimable:     205236 kB
Slab:             265320 kB
SReclaimable:     205236 kB
SUnreclaim:        60084 kB
KernelStack:        8616 kB
PageTables:        14784 kB
SecPageTables:         0 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    26734836 kB
Committed_AS:    3972368 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       14292 kB
VmallocChunk:    

## Preparation

In [ ]:
!cp '/content/drive/MyDrive/Final Project/Codes/animelist-goodbooks-recommendation/utils.py' .

In [ ]:
import pandas as pd
import sys
import surprise
import cornac
import pickle

from recommenders.utils.timer import Timer
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_random_split
from recommenders.evaluation.python_evaluation import (
    ndcg_at_k,
    precision_at_k,
    recall_at_k,
    catalog_coverage,
    distributional_coverage,
    diversity,
    novelty,
    serendipity,

)
from recommenders.models.surprise.surprise_utils import predict
from recommenders.utils.notebook_utils import store_metadata
from recommenders.models.cornac.cornac_utils import predict_ranking

from surprise import Dataset, NormalPredictor, Reader
from ranx import Qrels, Run, evaluate

from utils import compute_ranking_predictions

print(f"System version: {sys.version}")
print(f"Surprise version: {surprise.__version__}")

System version: 3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0]
Surprise version: 1.1.4


In [ ]:
# Model parameters
NUM_EPOCHS = 20
SEED=100

In [ ]:
DATA_PATH = '/content/drive/MyDrive/Final Project/Codes/animelist-goodbooks-recommendation/animelist/data_sample_split/'

In [ ]:
train_data = pd.read_csv(f'{DATA_PATH}/data_train_full.tsv', sep='\t')
test_data = pd.read_csv(f'{DATA_PATH}/data_test.tsv', sep='\t')

In [ ]:
train_data

,username,anime_id,anime_title,status,score,num_episodes_watched,updated_at,user_id,time_diff
0,----2o,5114,Fullmetal Alchemist: Brotherhood,completed,9,64,2014-04-27 03:24:30+00:00,1,NaN
1,----2o,1575,Code Geass: Hangyaku no Lelouch,completed,7,25,2014-04-27 03:29:08+00:00,1,0 days 00:04:38
2,----2o,6213,Toaru Kagaku no Railgun,completed,8,24,2015-05-20 19:07:56+00:00,1,388 days 15:38:48
3,----2o,22297,Fate/stay night: Unlimited Blade Works,completed,8,12,2015-05-22 05:01:43+00:00,1,1 days 09:53:47
4,----2o,2993,Rosario to Vampire,completed,6,13,2015-05-22 05:30:36+00:00,1,0 days 00:28:53
...,...,...,...,...,...,...,...,...,...
985965,zziky,39597,Sword Art Online: Alicization - War of Underworld,completed,7,12,2022-02-21 16:46:45+00:00,22988,0 days 00:00:07
985966,zziky,40540,Sword Art Online: Alicization - War of Underwo...,completed,7,11,2022-02-21 16:46:50+00:00,22988,0 days 00:00:05
985967,zziky,32093,Tanaka-kun wa Itsumo Kedaruge,completed,8,12,2022-02-21 16:47:08+00:00,22988,0 days 00:00:07
985968,zziky,16870,The Last: Naruto the Movie,completed,6,1,2022-02-21 16:47:31+00:00,22988,0 days 00:00:23


## Default Setting

### Singular Value Decomposition

In [ ]:
# A reader is still needed but only the rating_scale param is required.
reader = Reader(rating_scale=(1, 10))

# The columns must correspond to user id, item id and ratings (in that order).
train_set = Dataset.load_from_df(train_data[["user_id", "anime_id", "score"]], reader).build_full_trainset()
train_set

In [ ]:
svd = surprise.SVD(random_state=SEED, n_epochs=NUM_EPOCHS, verbose=True)

with Timer() as train_time:
    svd.fit(train_set)

print(f"Took {train_time.interval} seconds for training.")

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Took 10.517121310999983 seconds for training.


In [ ]:
with Timer() as test_time:
    all_predictions_svd = compute_ranking_predictions(
        svd, train_data, usercol="user_id", itemcol="anime_id", predcol="prediction", remove_seen=True
    )

print(f"Took {test_time.interval} seconds for prediction.")

  0%|          | 0/10000 [00:00<?, ?it/s]

Took 130.73176442099987 seconds for prediction.


In [ ]:
test_data

,username,anime_id,anime_title,status,score,num_episodes_watched,updated_at,user_id,time_diff
0,----2o,15051,Love Live! School Idol Project,completed,8,13,2017-04-15 01:51:18+00:00,1,0 days 00:01:11
1,----2o,19111,Love Live! School Idol Project 2nd Season,completed,8,13,2017-04-15 01:51:36+00:00,1,0 days 00:00:18
2,----2o,25157,Trinity Seven,completed,7,12,2017-04-15 19:02:55+00:00,1,0 days 17:11:19
3,----2o,20785,Mahouka Koukou no Rettousei,completed,8,26,2017-04-16 18:09:31+00:00,1,0 days 23:06:36
4,----2o,11235,Amagami SS+ Plus,completed,9,13,2017-04-17 00:13:21+00:00,1,0 days 06:03:50
...,...,...,...,...,...,...,...,...,...
246496,zziky,51096,Youkoso Jitsuryoku Shijou Shugi no Kyoushitsu ...,completed,6,13,2022-09-27 00:10:08+00:00,22988,56 days 18:03:49
246497,zziky,48561,Jujutsu Kaisen 0 Movie,completed,6,1,2022-10-05 01:55:49+00:00,22988,8 days 01:45:41
246498,zziky,48583,Shingeki no Kyojin: The Final Season Part 2,completed,9,12,2023-03-04 04:46:49+00:00,22988,115 days 05:57:44
246499,zziky,51535,Shingeki no Kyojin: The Final Season - Kankets...,completed,9,2,2023-11-12 05:30:18+00:00,22988,34 days 14:16:21


### BPR and Mpop

In [ ]:
eval_method = cornac.eval_methods.BaseMethod.from_splits(
  train_data=list(train_data[['user_id', 'anime_id', 'score']].itertuples(index=False)),
  test_data=list(test_data[['user_id', 'anime_id', 'score']].itertuples(index=False)),
  exclude_unknowns=True,
  verbose=True,
  seed=SEED,
)

rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 10000
Number of items = 1000
Number of ratings = 985970
Max rating = 10.0
Min rating = 6.0
Global mean = 8.1
---
Test data:
Number of users = 10000
Number of items = 1000
Number of ratings = 246501
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 10000
Total items = 1000


In [ ]:
bpr = cornac.models.BPR(
    max_iter=NUM_EPOCHS,
    verbose=True,
    seed=SEED
)
most_pop = cornac.models.MostPop()

In [ ]:
with Timer() as t:
    bpr.fit(eval_method.train_set)
print("Took {} seconds for training.".format(t))

In [ ]:
with Timer() as t:
    most_pop.fit(eval_method.train_set)
print("Took {} seconds for training.".format(t))

Took 0.0015 seconds for training.


In [ ]:
with Timer() as t:
    all_predictions_bpr = predict_ranking(bpr, train_data, usercol='user_id', itemcol='anime_id', remove_seen=True)
print("Took {} seconds for prediction.".format(t))

Took 21.9672 seconds for prediction.


In [ ]:
with Timer() as t:
    all_predictions_mpop = predict_ranking(most_pop, train_data, usercol='user_id', itemcol='anime_id', remove_seen=True)
print("Took {} seconds for prediction.".format(t))

Took 24.7639 seconds for prediction.


In [ ]:
ranking_metrics_svd = {
    'Precision@1' : precision_at_k(test_data, all_predictions_svd, col_user="user_id", col_item="anime_id", col_rating='score', k=1),
    'Precision@10' : precision_at_k(test_data, all_predictions_svd, col_user="user_id", col_item="anime_id", col_rating='score', k=10),
    'Precision@20' : precision_at_k(test_data, all_predictions_svd, col_user="user_id", col_item="anime_id", col_rating='score', k=20),
    'Recall@1' : recall_at_k(test_data, all_predictions_svd, col_user="user_id", col_item="anime_id", col_rating='score', k=1),
    'Recall@10' : recall_at_k(test_data, all_predictions_svd, col_user="user_id", col_item="anime_id", col_rating='score', k=10),
    'Recall@20' : recall_at_k(test_data, all_predictions_svd, col_user="user_id", col_item="anime_id", col_rating='score', k=20),
    'NDCG@1' : ndcg_at_k(test_data, all_predictions_svd, col_user="user_id", col_item="anime_id", col_rating='score', k=1),
    'NDCG@10' : ndcg_at_k(test_data, all_predictions_svd, col_user="user_id", col_item="anime_id", col_rating='score', k=10),
    'NDCG@20' : ndcg_at_k(test_data, all_predictions_svd, col_user="user_id", col_item="anime_id", col_rating='score', k=20)
}

In [ ]:
ranking_metrics_bpr = {
    'Precision@1' : precision_at_k(test_data, all_predictions_bpr, col_user="user_id", col_item="anime_id", col_rating='score', k=1),
    'Precision@10' : precision_at_k(test_data, all_predictions_bpr, col_user="user_id", col_item="anime_id", col_rating='score', k=10),
    'Precision@20' : precision_at_k(test_data, all_predictions_bpr, col_user="user_id", col_item="anime_id", col_rating='score', k=20),
    'Recall@1' : recall_at_k(test_data, all_predictions_bpr, col_user="user_id", col_item="anime_id", col_rating='score', k=1),
    'Recall@10' : recall_at_k(test_data, all_predictions_bpr, col_user="user_id", col_item="anime_id", col_rating='score', k=10),
    'Recall@20' : recall_at_k(test_data, all_predictions_bpr, col_user="user_id", col_item="anime_id", col_rating='score', k=20),
    'NDCG@1' : ndcg_at_k(test_data, all_predictions_bpr, col_user="user_id", col_item="anime_id", col_rating='score', k=1),
    'NDCG@10' : ndcg_at_k(test_data, all_predictions_bpr, col_user="user_id", col_item="anime_id", col_rating='score', k=10),
    'NDCG@20' : ndcg_at_k(test_data, all_predictions_bpr, col_user="user_id", col_item="anime_id", col_rating='score', k=20)
}

In [ ]:
ranking_metrics_mpop = {
    'Precision@1' : precision_at_k(test_data, all_predictions_mpop, col_user="user_id", col_item="anime_id", col_rating='score', k=1),
    'Precision@10' : precision_at_k(test_data, all_predictions_mpop, col_user="user_id", col_item="anime_id", col_rating='score', k=10),
    'Precision@20' : precision_at_k(test_data, all_predictions_mpop, col_user="user_id", col_item="anime_id", col_rating='score', k=20),
    'Recall@1' : recall_at_k(test_data, all_predictions_mpop, col_user="user_id", col_item="anime_id", col_rating='score', k=1),
    'Recall@10' : recall_at_k(test_data, all_predictions_mpop, col_user="user_id", col_item="anime_id", col_rating='score', k=10),
    'Recall@20' : recall_at_k(test_data, all_predictions_mpop, col_user="user_id", col_item="anime_id", col_rating='score', k=20),
    'NDCG@1' : ndcg_at_k(test_data, all_predictions_mpop, col_user="user_id", col_item="anime_id", col_rating='score', k=1),
    'NDCG@10' : ndcg_at_k(test_data, all_predictions_mpop, col_user="user_id", col_item="anime_id", col_rating='score', k=10),
    'NDCG@20' : ndcg_at_k(test_data, all_predictions_mpop, col_user="user_id", col_item="anime_id", col_rating='score', k=20)
}

In [ ]:
pd.DataFrame({"SVD": ranking_metrics_svd, "BPR":ranking_metrics_bpr, "Mpop": ranking_metrics_mpop})

,SVD,BPR,Mpop
Precision@1,0.095500,0.190300,0.187700
Precision@10,0.079550,0.120400,0.119990
Precision@20,0.075055,0.105200,0.105045
Recall@1,0.003806,0.009897,0.009820
Recall@10,0.031039,0.058876,0.058658
Recall@20,0.057431,0.098697,0.099511
NDCG@1,0.095500,0.190300,0.187700
NDCG@10,0.084053,0.137172,0.135864
NDCG@20,0.085497,0.134924,0.134408


### Evaluation using Ranx

In [ ]:
max_item = all_predictions_mpop['prediction'].max()

In [ ]:
all_predictions_mpop['prediction'] = all_predictions_mpop['prediction']/max_item

In [ ]:
test_data['user_id'] = test_data['user_id'].astype(str)
test_data['anime_id'] = test_data['anime_id'].astype(str)

all_predictions_svd['user_id'] = all_predictions_svd['user_id'].astype(str)
all_predictions_svd['anime_id'] = all_predictions_svd['anime_id'].astype(str)

all_predictions_bpr['user_id'] = all_predictions_bpr['user_id'].astype(str)
all_predictions_bpr['anime_id'] = all_predictions_bpr['anime_id'].astype(str)

all_predictions_mpop['user_id'] = all_predictions_mpop['user_id'].astype(str)
all_predictions_mpop['anime_id'] = all_predictions_mpop['anime_id'].astype(str)

In [ ]:
qrels = Qrels.from_df(
    df=test_data,
    q_id_col="user_id",
    doc_id_col="anime_id",
    score_col="score",
)

run_svd = Run.from_df(
    df=all_predictions_svd,
    q_id_col="user_id",
    doc_id_col="anime_id",
    score_col="prediction",
)

run_bpr = Run.from_df(
    df=all_predictions_bpr,
    q_id_col="user_id",
    doc_id_col="anime_id",
    score_col="prediction",
)

run_mpop = Run.from_df(
    df=all_predictions_mpop,
    q_id_col="user_id",
    doc_id_col="anime_id",
    score_col="prediction",
)

In [ ]:
rank_all_df = pd.DataFrame({'SVD': evaluate(qrels, run_svd, ["mrr", "ndcg@10", "recall@10", "precision@10", "hit_rate@10"]),
                            'BPR': evaluate(qrels, run_bpr, ["mrr", "ndcg@10", "recall@10", "precision@10", "hit_rate@10"]),
                            'Mpop': evaluate(qrels, run_mpop, ["mrr", "ndcg@10", "recall@10", "precision@10", "hit_rate@10"])})

In [ ]:
rank_all_df

,SVD,BPR,Mpop
mrr,0.203378,0.324593,0.321793
ndcg@10,0.082481,0.126941,0.125854
recall@10,0.030963,0.058876,0.058658
precision@10,0.080190,0.120400,0.119990
hit_rate@10,0.434900,0.617600,0.614500


## Parameters from Hyperparameter Opt

### Singular Value Decomposition

In [ ]:
# A reader is still needed but only the rating_scale param is required.
reader = Reader(rating_scale=(1, 10))

# The columns must correspond to user id, item id and ratings (in that order).
train_set = Dataset.load_from_df(train_data[["user_id", "anime_id", "score"]], reader).build_full_trainset()
train_set

In [ ]:
svd = surprise.SVD(random_state=SEED,
                         n_epochs=10,
                         verbose=False,
                         n_factors=200,
                         lr_all = 0.0045,
                         reg_all = 0.0014
                         )

with Timer() as train_time:
    svd.fit(train_set)

print(f"Took {train_time.interval} seconds for training.")

Took 10.98990552600003 seconds for training.


In [ ]:
with Timer() as test_time:
    all_predictions_svd = compute_ranking_predictions(
        svd, train_data, usercol="user_id", itemcol="anime_id", predcol="prediction", remove_seen=True
    )

print(f"Took {test_time.interval} seconds for prediction.")

  0%|          | 0/10000 [00:00<?, ?it/s]

Took 132.72843948799994 seconds for prediction.


### BPR and Mpop

In [ ]:
eval_method = cornac.eval_methods.BaseMethod.from_splits(
  train_data=list(train_data[['user_id', 'anime_id', 'score']].itertuples(index=False)),
  test_data=list(test_data[['user_id', 'anime_id', 'score']].itertuples(index=False)),
  exclude_unknowns=True,
  verbose=True,
  seed=SEED,
)

rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 10000
Number of items = 1000
Number of ratings = 985970
Max rating = 10.0
Min rating = 6.0
Global mean = 8.1
---
Test data:
Number of users = 10000
Number of items = 1000
Number of ratings = 246501
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 10000
Total items = 1000


In [ ]:
bpr = cornac.models.BPR(
    k=140,
    max_iter=20,
    learning_rate=0.0364,
    lambda_reg=0.0015,
    verbose=True,
    seed=SEED
  )
most_pop = cornac.models.MostPop()

In [ ]:
with Timer() as t:
    bpr.fit(eval_method.train_set)
print("Took {} seconds for training.".format(t))

  0%|          | 0/20 [00:00<?, ?it/s]

Optimization finished!
Took 13.1329 seconds for training.


In [ ]:
with Timer() as t:
    most_pop.fit(eval_method.train_set)
print("Took {} seconds for training.".format(t))

Took 0.0298 seconds for training.


In [ ]:
with Timer() as t:
    all_predictions_bpr = predict_ranking(bpr, train_data, usercol='user_id', itemcol='anime_id', remove_seen=True)
print("Took {} seconds for prediction.".format(t))

Took 18.0634 seconds for prediction.


In [ ]:
with Timer() as t:
    all_predictions_mpop = predict_ranking(most_pop, train_data, usercol='user_id', itemcol='anime_id', remove_seen=True)
print("Took {} seconds for prediction.".format(t))

Took 16.3784 seconds for prediction.


In [ ]:
PREDICTION_PATH = '/content/drive/MyDrive/Final Project/Codes/animelist-goodbooks-recommendation/animelist/predictions'
pickle.dump(all_predictions_svd, open(f'{PREDICTION_PATH}/predictions_svd_opt.pkl', 'wb'))
pickle.dump(all_predictions_mpop, open(f'{PREDICTION_PATH}/predictions_pop_opt.pkl', 'wb'))
pickle.dump(all_predictions_bpr, open(f'{PREDICTION_PATH}/predictions_bpr_opt.pkl', 'wb'))

### Evaluation

In [ ]:
max_item = all_predictions_mpop['prediction'].max()

In [ ]:
all_predictions_mpop['prediction'] = all_predictions_mpop['prediction']/max_item

In [ ]:
test_data['user_id'] = test_data['user_id'].astype(str)
test_data['anime_id'] = test_data['anime_id'].astype(str)

all_predictions_svd['user_id'] = all_predictions_svd['user_id'].astype(str)
all_predictions_svd['anime_id'] = all_predictions_svd['anime_id'].astype(str)

all_predictions_bpr['user_id'] = all_predictions_bpr['user_id'].astype(str)
all_predictions_bpr['anime_id'] = all_predictions_bpr['anime_id'].astype(str)

all_predictions_mpop['user_id'] = all_predictions_mpop['user_id'].astype(str)
all_predictions_mpop['anime_id'] = all_predictions_mpop['anime_id'].astype(str)

In [ ]:
qrels = Qrels.from_df(
    df=test_data,
    q_id_col="user_id",
    doc_id_col="anime_id",
    score_col="score",
)

run_svd = Run.from_df(
    df=all_predictions_svd,
    q_id_col="user_id",
    doc_id_col="anime_id",
    score_col="prediction",
)

run_bpr = Run.from_df(
    df=all_predictions_bpr,
    q_id_col="user_id",
    doc_id_col="anime_id",
    score_col="prediction",
)

run_mpop = Run.from_df(
    df=all_predictions_mpop,
    q_id_col="user_id",
    doc_id_col="anime_id",
    score_col="prediction",
)

In [ ]:
rank_all_df = pd.DataFrame({'SVD': evaluate(qrels, run_svd, ["mrr", "ndcg@10", "recall@10", "precision@10", "hit_rate@10"]),
                            'BPR': evaluate(qrels, run_bpr, ["mrr", "ndcg@10", "recall@10", "precision@10", "hit_rate@10"]),
                            'Mpop': evaluate(qrels, run_mpop, ["mrr", "ndcg@10", "recall@10", "precision@10", "hit_rate@10"])})

/usr/local/lib/python3.11/dist-packages/ranx/metrics/reciprocal_rank.py:29: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  scores[i] = _reciprocal_rank(qrels[i], run[i], k, rel_lvl)


In [ ]:
rank_all_df

,SVD,BPR,Mpop
mrr,0.198941,0.452208,0.321793
ndcg@10,0.077246,0.208092,0.125854
recall@10,0.029204,0.105769,0.058658
precision@10,0.074830,0.197330,0.119990
hit_rate@10,0.425200,0.769400,0.614500
